# OWSM finetuning for Spoken Language Understanding task
This Jupyter notebook provides a step-by-step guide on using the ESPnetEZ module to finetune owsm model. In this demonstration, we will leverage the `SLURP` dataset (intent classification task) to finetune an OWSM model for SLU task. 

This demo will focus on how to add new tokens to the pre-traied OWSM model for the new task, intent classification. For the other sections such as data preparation, training, and evaluation, please refer to the other notebooks.

In this notebook, we assume that you have already downloaded the `SLUPR` dataset, and created the dump file using the recipe. If you haven't, and you have never used recipe prepared in espnet, you can refer to the data preparation section of `train_from_scratch.ipynb` or `finetune_owsm.ipynb` notebooks in ASR demos.

Author: Masao Someki [@Masao-Someki](https://github.com/Masao-Someki)

First, let's install espnet if you haven't it.

In [ ]:
!pip install -U espnet

Then import necessary libraries and set several hyper parameters.

In [ ]:
import torch
import numpy as np
import librosa
from pathlib import Path
from espnet2.layers.create_adapter_fn import create_lora_adapter
import argparse

import espnetez as ez


FINETUNE_MODEL = "espnet/owsm_v3.1_ebf_base"

DATA_PATH = "./data"
DUMP_DIR = "./dump/raw"
STATS_DIR = "./exp/stats_owsm"

ADDITIONAL_SPECIAL_TOKENS = [
    "<intent>"
]

## Add new token to the pre-trained OWSM tokenizer

Here we will add new `<intent>` token to the pre-trained OWSM tokenizer. We prepared the `add_special_tokens` function from `espnet2.preprocess` module to add new tokens to the pretrained tokenizer, converter, and the Embedding layer.

We will use the Embedding layer for the `build_model_fn` function to replace the pre-trained Embedding layer with the new one.

In [ ]:
from espnet2.bin.s2t_inference import Speech2Text
pretrained_model = Speech2Text.from_pretrained(
    FINETUNE_MODEL,
    # category_sym="<en>",
    beam_size=10,
    device="cpu"
)
tokenizer = pretrained_model.tokenizer
converter = pretrained_model.converter

# Add new <intent_cls> token after ST-related tokens
tokenizer, converter, _ = ez.preprocess.add_special_tokens(
    tokenizer, converter, pretrained_model.s2t_model.decoder.embed[0],
    ADDITIONAL_SPECIAL_TOKENS, insert_after="<st_zho>"
)

In [ ]:
# And load configuration of pre-trained model. before deleting it.
training_config = vars(pretrained_model.s2t_train_args)
del pretrained_model

## Data preparation

Since we need to create text with the new token `<intent>`, we need to modify the dump file generated by recipe. We want to modify the `text` to the following format: 

`<eng><intent><notimestamps> intent`

To chieve this, we need to write a custom dataset class and `data_info` with functions.

In [ ]:
class CustomDataset:
    def __init__(self, data_path, is_train=True):
        self.data_path = data_path
        if is_train:
            data_path = f"{data_path}/train"
        else:
            data_path = f"{data_path}/devel"
        
        self.data = {}
        with open(f"{data_path}/wav.scp", "r") as f:
            for line in f.readlines():
                audio_id, audio_path = line.strip().split(maxsplit=1)
                self.data[audio_id] = {
                    'audio_path': audio_path
                }

        with open(f"{data_path}/transcript", "r") as f:
            for line in f.readlines():
                audio_id, translated = line.strip().split(maxsplit=1)
                self.data[audio_id]['transcript'] = translated
        
        with open(f"{data_path}/text", "r") as f:
            for line in f.readlines():
                audio_id, intent, _ = line.strip().split(maxsplit=2)
                self.data[audio_id]['intent'] = intent
        
        self.keys = list(self.data.keys())
    
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, idx):
        return {
            'audio_path': self.data[idx]['audio_path'],
            'intent': self.data[idx]['intent'],
            'transcript': self.data[idx]['transcript']
        }


In [ ]:
def tokenize(text):
    return np.array(converter.tokens2ids(tokenizer.text2tokens(text)))

data_info = {
    "speech": lambda d : librosa.load(d['audio_path'], sr=16000)[0],
    "text": lambda d : tokenize(f"<eng><intent><notimestamps>{d['intent']}"),
    "text_prev": lambda d : tokenize("<na>"),
    "text_ctc": lambda d : tokenize(d['transcript'].lower()),
} 

## Model preparation

Let's prepare the `build_model_fn` function for the Trainer. We will use the OWSM-v3.1-ebf-base model, which has approxmately 100M parameters.

Please note that we cannot initiate the model outside of a function. When we try to initialize the model with multi-GPU environment, we need to initialize model for each GPUs. For this reason, it is easier to run model initialization funtion for each GPU rather than copying the model. So we need to write initialization code inside a function.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def build_model_fn(args):
    pretrained_model = Speech2Text.from_pretrained(
        FINETUNE_MODEL,
        beam_size=10,
    )
    model = pretrained_model.s2t_model
    model.train()
    print(f'Trainable parameters: {count_parameters(model)}')
    
    # Add new <intent> token
    _, _, new_embedding = ez.preprocess.add_special_tokens(
        pretrained_model.tokenizer,
        pretrained_model.converter,
        model.decoder.embed[0],
        ADDITIONAL_SPECIAL_TOKENS,
        insert_after="<st_zho>"
    )
    new_embedding.weight.requires_grad = True
    model.decoder.embed[0] = new_embedding
    # apply lora if you want.
    # create_lora_adapter(model, target_modules=LORA_TARGET)
    # print(f'Trainable parameters after LORA: {count_parameters(model)}')
    return model


## Training Configuration

Now let's set up the training configuration for the OWSM finetuning. Basically all configurations are the same as the OWSM training, but we will change some parameters for this finetuning.

In [ ]:
# Extract training config from the pretrained model.
finetune_config = ez.config.update_finetune_config(
    "s2t",
    training_config,
    args.config
)
finetune_config['multiple_iterator'] = False

In [ ]:
train_dataset = CustomDataset(data_path="./dump/raw", is_train=True)
dev_dataset = CustomDataset(data_path="./dump/raw", is_train=False)

train_dataset = ez.dataset.ESPnetEZDataset(train_dataset, data_info=data_info)
dev_dataset = ez.dataset.ESPnetEZDataset(dev_dataset, data_info=data_info)

## Training

Now we have everything prepared, we can start training the OWSM model for ST task.

In [ ]:
trainer = ez.Trainer(
    task="s2t",
    train_config=finetune_config,
    train_dataset=train_dataset,
    valid_dataset=dev_dataset,
    data_info=data_info,
    build_model_fn=build_model_fn,
    output_dir=EXP_DIR,
    stats_dir=STATS_DIR,
    ngpu=1,
)
trainer.collect_stats()
trainer.train()